In [40]:
import pandas as pd
import numpy as np 

from datetime import timedelta
from itertools import compress

In [41]:
df = pd.read_excel("./data/ESO 2020Q1.xls", header = 2)

In [42]:
# create dictionary of trimmed incident type numbers

incident_types = {
    100 : "Fire", 
    200 : "Overpressure explosion, overheat - no fire", 
    300 : "Rescue and EMS incidents", 
    400 : "Hazardous Conditions - no fire", 
    500 : "Service Call", 
    600 : "Good intent calls", 
    700 : "False alarms and false calls", 
    800 : "Severe Weather and natural disaster", 
    900 : "Special incident type"
}

In [43]:
df.head()

,Incident Number,Alarm Date,Station,Actions Taken,Aid Given Or Received,Address,Apparatus Name,PSAP Received Date,Dispatched Date,En Route Date,...,Clear Date,Personnel Count,Priority,Type of Response Delay,Is Cancelled Prior To Arrival,Incident Type,Incident Type Code,Shift,Longitude,Latitude
0,20-0324,2020-02-13 15:26:37,1-3,Assist physically disabled,None,"1237 GERDES, Camano Island, Washington",AID103,2020-02-13 15:25:56,2020-02-13 15:26:37,2020-02-13 15:26:58,...,2020-02-13 15:42:29,2.0,Non-Emergent,NaN,NaN,Assist invalid,554.0,Day,-122.471857,48.255921
1,20-0345,2020-02-17 02:15:28,1-2,Provide first aid & check for injuries,None,"1467 RIDGE, Camano Island, Washington",AID103,2020-02-17 02:13:58,2020-02-17 02:15:28,2020-02-17 02:17:06,...,2020-02-17 02:56:01,2.0,Emergent,NaN,NaN,"EMS call, excluding vehicle accident with injury",321.0,B Shift,-122.487304,48.157416
2,20-0345,2020-02-17 02:15:28,1-2,Provide first aid & check for injuries,None,"1467 RIDGE, Camano Island, Washington",MED102,2020-02-17 02:13:58,2020-02-17 02:15:28,2020-02-17 02:16:08,...,2020-02-17 02:56:01,2.0,Emergent,NaN,NaN,"EMS call, excluding vehicle accident with injury",321.0,B Shift,-122.487304,48.157416
3,20-0371,2020-02-21 19:32:35,1-4,Provide basic life support (BLS) Transport per...,None,"731 CHERYL ANN, Camano Island, Washington",AID103,2020-02-21 19:31:50,2020-02-21 19:32:35,2020-02-21 19:33:32,...,2020-02-21 21:03:42,2.0,Emergent,None/No Delay,NaN,"EMS call, excluding vehicle accident with injury",321.0,B Shift,-122.523413,48.186637
4,20-0371,2020-02-21 19:32:35,1-4,Provide basic life support (BLS) Transport per...,None,"731 CHERYL ANN, Camano Island, Washington",E14,2020-02-21 19:31:50,2020-02-21 19:32:35,2020-02-21 19:33:51,...,2020-02-21 19:57:37,3.0,Emergent,None/No Delay,NaN,"EMS call, excluding vehicle accident with injury",321.0,B Shift,-122.523413,48.186637


In [44]:
df.dtypes

Incident Number                          object
Alarm Date                       datetime64[ns]
Station                                  object
Actions Taken                            object
Aid Given Or Received                    object
Address                                  object
Apparatus Name                           object
PSAP Received Date               datetime64[ns]
Dispatched Date                  datetime64[ns]
En Route Date                    datetime64[ns]
Arrival Date                     datetime64[ns]
Clear Date                       datetime64[ns]
Personnel Count                         float64
Priority                                 object
Type of Response Delay                   object
Is Cancelled Prior To Arrival            object
Incident Type                            object
Incident Type Code                      float64
Shift                                    object
Longitude                               float64
Latitude                                

In [45]:
df.isnull().sum()

Incident Number                     2
Alarm Date                          2
Station                             2
Actions Taken                       2
Aid Given Or Received               2
Address                             2
Apparatus Name                      2
PSAP Received Date                  2
Dispatched Date                     2
En Route Date                      40
Arrival Date                      258
Clear Date                          2
Personnel Count                     2
Priority                            2
Type of Response Delay           1202
Is Cancelled Prior To Arrival    1063
Incident Type                       2
Incident Type Code                  2
Shift                               2
Longitude                          14
Latitude                           14
dtype: int64

In [46]:
df = df[df["Incident Number"].isnull() == False].copy()

In [47]:
# calculate time lapse intervals for incidents

df["Turn Out Time Elapsed"] = df["En Route Date"] - df["Dispatched Date"]

df["Travel Time Elapsed"] = df["Arrival Date"] - df["En Route Date"]

df["Response Time Elapsed"] = df["Arrival Date"] - df["Dispatched Date"]

df["Onsite Time Elapsed"] = df["Clear Date"] - df["Arrival Date"]

df["Total Incident Time Elapsed"] = sum([df["Turn Out Time Elapsed"], df["Travel Time Elapsed"], df["Onsite Time Elapsed"]])


In [48]:
# get trimmed incident type code

df["Incident Type Code Trimmed"] = [int(str(df["Incident Type Code"][i])[0] + "00") for i in df.index]

df["Incident Type Trimmed"] = [incident_types[i] for i in df["Incident Type Code Trimmed"]]

In [49]:
# note turn out times that meet goal 
df["Unit Turn Out Goal Met"] = ["YES" if df["Response Time Elapsed"][i] <= timedelta(minutes = 9, seconds = 30) else "NO" for i in df.index]

In [50]:
# adding columns for elapsed time that is only in seconds 
# Tableau is dumb, and wigs out with elapses of time longer than 24 hours 
# work arounds need elapsed time to be in seconds in the dataset

df["Turn Out Time Elapsed (seconds)"] = [pd.Timedelta(df["Turn Out Time Elapsed"][n]).seconds for n in df.index]

df["Travel Time Elapsed (seconds)"] = [pd.Timedelta(df["Travel Time Elapsed"][n]).seconds for n in df.index]

df["Response Time Elapsed (seconds)"] = [pd.Timedelta(df["Response Time Elapsed"][n]).seconds for n in df.index]

df["Onsite Time Elapsed (seconds)"] = [pd.Timedelta(df["Onsite Time Elapsed"][n]).seconds for n in df.index]

df["Total Incident Time Elapsed (seconds)"] = [pd.Timedelta(df["Total Incident Time Elapsed"][n]).seconds for n in df.index]

In [51]:
# identify unit number with fastest response time 
fos_units = {}

for i in df['Incident Number']: 
    time_list = list(df[df['Incident Number'] == i]['Turn Out Time Elapsed (seconds)'])
    bool_list = [not b for b in list(np.isnan(time_list))]
    time_list = list(compress(time_list, bool_list))
    try: 
        min_time = min(time_list)
    except: 
        min_time = None
    unit = df[(df["Turn Out Time Elapsed (seconds)"] == min_time) & (df['Incident Number'] == i)]["Apparatus Name"].to_numpy()
    fos_units[i] = unit

In [52]:
fos_units['20-0324'][0]

'AID103'

In [72]:
# note first on scene unit

filler = [0 for i in df.index]

df["FOS unit"] = filler

for i in df.index: 
    inum = df.loc[i, 'Incident Number']
    try: 
        df.loc[i, 'FOS unit'] = fos_units[inum][0]
    except: 
        df.loc[i, 'FOS unit'] = np.nan

In [75]:
# note times of first on scene for each incident

df["FOS Turn Out (seconds)"] = filler
df["FOS Response Time Elapsed (seconds)"] = filler
df["FOS Travel Time Elapsed (seconds)"] = filler

for i in df.index: 
    inum = df.loc[i, 'Incident Number']
    try: 
        df.loc[i, 'FOS Turn Out (seconds)'] = df[(df['Incident Number'] == inum) & 
                                             (df['Apparatus Name'] == fos_units[inum][0])]['Turn Out Time Elapsed (seconds)'].to_numpy()[0]
    except: 
        df.loc[i, 'FOS Turn Out (seconds)'] = np.nan
    try: 
        df.loc[i, 'FOS Response Time Elapsed (seconds)'] = df[(df['Incident Number'] == inum) & 
                                                         (df['Apparatus Name'] == fos_units[inum][0])]['Response Time Elapsed (seconds)'].to_numpy()[0]
    except: 
        df.loc[i, 'FOS Response Time Elapsed (seconds)'] = np.nan
    try: 
        df.loc[i, 'FOS Travel Time Elapsed (seconds)'] = df[(df['Incident Number'] == inum) & 
                                                       (df['Apparatus Name'] == fos_units[inum][0])]['Travel Time Elapsed (seconds)'].to_numpy()[0]    
    except: 
        df.loc[i, 'FOS Travel Time Elapsed (seconds)'] = np.nan


In [79]:
# turn out goal met per incident
goal = 9 * 60 + 30
df["Incident Turn Out Goal Met"] = ["YES" if df["FOS Response Time Elapsed (seconds)"][i] <= goal else "NO" for i in df.index]

In [81]:
# export

df.to_excel("./data/ESO 2020Q1_cleaned.xlsx")

In [102]:
emergent_goal_met = sum(df[df['Priority'] == 'Emergent']['FOS Response Time Elapsed (seconds)'] <= goal)
emergent_goal_unmet = sum(df[df['Priority'] == 'Emergent']['FOS Response Time Elapsed (seconds)'] > goal)
emergent_nulls = len(df[(df['Priority'] == 'Emergent') & (df['FOS Response Time Elapsed (seconds)'].isnull())]['FOS Response Time Elapsed (seconds)'])

emergent_total = len(df[df['Priority'] == 'Emergent']['FOS Response Time Elapsed (seconds)'])

In [127]:
emergent_goal_met + emergent_goal_unmet + emergent_nulls

881

In [129]:
emergent_total

881

In [128]:
emergent_goal_unmet/emergent_total

0.36095346197502837

In [130]:
emergent_goal_met/emergent_total

0.5017026106696936

In [131]:
emergent_goal_met/(emergent_total - emergent_nulls)

0.5815789473684211

In [132]:
emergent_goal_unmet/(emergent_total - emergent_nulls)

0.41842105263157897

In [135]:
len(df[df['Priority'] == 'Emergent']['Incident Number'].unique())

344

In [136]:
len(df[df['Priority'] == 'Emergent']['Incident Number'])

881